In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern01.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern04.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern03.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern06.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern05.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern07.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern02.txt
/kaggle/input/my-private-datasets/CVBooks/SoT_CV-Jr_2018_rus.txt
/kaggle/input/my-private-datasets/CVBooks/ollection_of_quotes_2015_rus.txt
/kaggle/input/my-private-datasets/CVBooks/rus/3rdAlt_2017.txt
/kaggle/input/my-private-datasets/CVBooks/rus/7h_1998.txt
/kaggle/input/my-private-datasets/CVBooks/rus/LonP_2008.txt
/kaggle/input/my-private-datasets/CVBooks/rus/7h_fam_2007.txt
/kaggle/input/my-private-datasets/CVBooks/rus/PredRes_2010.txt
/kaggle/input/my-private-datasets/CVBooks/rus/8h_2017.txt
/kaggle/input/my-private-datasets/C

- сравнить NER
- опробовать разные варианты dependencies
- можно делать это на саммаризованых текстах

In [2]:
# tmp = pd.read_csv("/kaggle/input/my-private-datasets/graph_news/news_with_summarization.csv", index_col=0, parse_dates=['date'])
# tmp[tmp.date.dt.year == 2022].groupby(tmp.date.dt.month).text.count().iloc[:-1].mean()
# # Average number of news items per month: 1776

In [3]:
SAMPLE_SIZE = 1000
pd.set_option('display.max_colwidth', None)

df_sample = pd.read_csv("/kaggle/input/my-private-datasets/graph_news/news_with_summarization.csv", index_col=0).sample(SAMPLE_SIZE)
df_sample.rename(columns={"IlyaGusev__rut5_base_headline_gen_telegram": "summary"}, inplace=True)
df_sample["text"] = df_sample.text.str.replace(r"[^\x20-\xFFа-яА-ЯёЁ№\n]+", "", regex=True) # Clean ⚡️🎾❗️🌏...
df_sample

,date,text,summary
165028,2022-08-31T18:03:20,"Глава МИД Германии Бербок: ""Я обещала людям Украины, что мы будем с вами столько, сколько нужно, и я хочу сдержать это обещание. Неважно, что думают мои немецкие избиратели""","Бербок: Я обещала людям Украины, что мы будем с вами столько, сколько нужно"
109813,2021-09-20T11:25:08,"Думские выборы прошли в соответствии с российскими законами и демократическими стандартами, были открытыми и конкурентными, заявила миссия наблюдателей СНГ",миссия наблюдателей СНГ: Думские выборы прошли в соответствии с законами и демократическими стандартами
2642,2019-10-10T02:13:32,"Россиянка Юзик, задержанная ранее в Иране и вылетевшая 10 октября в Москву, чувствует себя нормально - посольство РФ в Тегеране РИА Новости",Посольство РФ: задержанная в Иране россиянка чувствует себя нормально
137131,2022-02-24T15:35:07,"Некоторые украинские военные в ДНР сдаются, увидев бегство своих командиров, сказал РИА Новости Басурин","Басурин: Некоторые украинские военные в ДНР сдаются, увидев бегство своих командиров"
86234,2021-05-14T11:26:10,"Крыша 8-этажного жилого дома горит в Екатеринбурге (Шейнкмана 19), идет эвакуация жильцов",Крыша 8-этажного жилого дома горит в Екатеринбурге (Шейнкмана 19)
...,...,...,...
10160,2020-02-26T16:47:59,"Вопрос о переносе ПМЭФ из-за коронавируса пока не обсуждается, сообщила Голикова",Голикова: вопрос о переносе ПМЭФ из-за коронавируса пока не обсуждается
113889,2021-10-13T19:43:55,"Половина французов ставит Макрону ""двойку"": 52% считают его действия на посту президента неудовлетворительными, 22% одобряют его работу, а 26% говорят, что еще рано давать оценки, свидетельствует опрос Elabe для BFM TV","Половина французов ставит Макрону ""двойку"""
124828,2021-12-21T14:02:55,"Шойгу: в России создан первый авиаполк истребителей МиГ-31 с гиперзвуковыми ракетами ""Кинжал""","В России создан первый авиаполк истребителей МиГ-31 с гиперзвуковыми ракетами ""Кинжал"""
148012,2022-04-18T05:01:24,"ВВП Китая в I квартале 2022 вырос на 4,8%, сообщило статбюро. Власти ожидают, что темпы роста по итогам года составят около 5,5%.\n\nПо итогам 2021, несмотря на многочисленные локальные вспышки коронавируса, проблемы на рынке недвижимости и энергетический кризис, ВВП Китая вырос на 8,1% выше официального прогноза в 6%.","ВВП Китая в I квартале 2022 вырос на 4,8%"


## NER (compare spacy/stanza/natasha) ##

### NER normalization conclusions

Выводы по NER нормализации:
- Spacy/Stanza позволяют проводить только потокенную нормализацию, которая ужасно работает на составных NER например мы получаем 'государственный дума', 'общество защита животное' и т.д.
- В свою очередь Natasha позволяет проводить номалицацию с о учетом синтаксиса, например "Обществу защиты животных" -> "Общество защиты животных", при этом, Natasha хуже Spacy/Stanza по метрикам выделения самих NER (очень часто в простых случаях с односложными токенами). -> Итого, либо использовать для выделения NER ансамбль Natasha/Stanza, нормализовать Natasha все что она извлекла, а все оставшее нормализовать потокенной лемматизацией Stanza, либо научиться нормализовать через Natasha любые фразы (с предварительной конвертацией их в необходимый формат); 1-ый способ прост и может быть реализован очень быстро, 2-й по необходимости в будущем. 

### Ansamble Spacy/Natasha NER extraction and normalization

In [4]:
!pip -q install stanza
!pip -q install natasha

In [5]:
import stanza
#nlp = stanza.Pipeline(lang='ru')
# nlp_stanza = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma,ner,depparse')
nlp_stanza = stanza.Pipeline(lang='ru', processors='tokenize,ner')

In [6]:
import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [7]:
def normalize_ners(text):
    
    stanza_ners = nlp_stanza(text).ents
    
    natasha_doc = Doc(text)
    natasha_doc.segment(segmenter)
    natasha_doc.tag_morph(morph_tagger)
    natasha_doc.tag_ner(ner_tagger)
    natasha_ners_text = tuple([span.text for span in natasha_doc.spans])
    only_stanza_ners = [stanza_ner for stanza_ner in stanza_ners 
                        if stanza_ner.text not in natasha_ners_text]
    for ent in only_stanza_ners:
        id_start, id_stop = None, None
        for i in range(len(natasha_doc.tokens)):
            if natasha_doc.tokens[i].start == ent.start_char:
                id_start = i
                char_start = natasha_doc.tokens[i].start
            if natasha_doc.tokens[i].stop >= ent.end_char:
                id_stop = i
                char_stop = natasha_doc.tokens[i].stop
                break
        if id_start is not None and id_stop is not None:
            
            natasha_doc.spans.append(
                natasha.doc.DocSpan(start=char_start, 
                                    stop=char_stop, 
                                    type=ent.type,
                                    text=text[char_start:char_stop],
                                    tokens=natasha_doc.tokens[id_start:id_stop + 1]))
        else:
            print(f"Info message: Not compatible tokens:")
            print(ent)
            print("\n".join(map(str, natasha_doc.tokens)))
            print("\n".join(map(str, natasha_doc.spans)))
                                     
    for span in natasha_doc.spans:
        span.normalize(morph_vocab)
        
    return natasha_doc.spans

In [8]:
# # unit-test
# timeit_sample = "Футболиста сборной России Андрея Мостового из-за неблагоприятного результата ПЦР-тестирования не будет на чемпионате Европы, сообщается в канале команды в Telegram"
# print(nlp_stanza(timeit_sample).ents)
# normalize_ners(timeit_sample

In [9]:
%%time
df_sample['ners_norm_full'] = df_sample.text.map(lambda x: tuple(set(map(lambda y: (y.text, y.normal), normalize_ners(x)))))
df_sample['ners_norm_summ'] = df_sample.summary.map(lambda x: tuple(set(map(lambda y: (y.text, y.normal), normalize_ners(x)))))

Info message: Not compatible tokens:
{
  "text": "на-Амуре",
  "type": "LOC",
  "start_char": 241,
  "end_char": 249
}
DocToken(stop=11, text='Хабаровская', pos='ADJ', feats=<Nom,Pos,Fem,Sing>)
DocToken(start=12, stop=18, text='газета', pos='NOUN', feats=<Inan,Nom,Fem,Sing>)
DocToken(start=19, stop=20, text='"', pos='PUNCT')
DocToken(start=20, stop=29, text='Приамурье', pos='PROPN', feats=<Inan,Nom,Neut,Sing>)
DocToken(start=29, stop=30, text='"', pos='PUNCT')
DocToken(start=31, stop=40, text='объяснила', pos='VERB', feats=<Perf,Fem,Ind,Sing,Past,Fin,Act>)
DocToken(start=41, stop=46, text='выход', pos='NOUN', feats=<Inan,Acc,Masc,Sing>)
DocToken(start=47, stop=53, text='номера', pos='NOUN', feats=<Inan,Gen,Masc,Sing>)
DocToken(start=54, stop=56, text='на', pos='ADP')
DocToken(start=57, stop=58, text='"', pos='PUNCT')
DocToken(start=58, stop=69, text='тарабарском', pos='ADJ', feats=<Loc,Pos,Masc,Sing>)
DocToken(start=70, stop=75, text='языке', pos='NOUN', feats=<Inan,Loc,Masc,Sing>)
Doc

In [10]:
df_sample

,date,text,summary,ners_norm_full,ners_norm_summ
165028,2022-08-31T18:03:20,"Глава МИД Германии Бербок: ""Я обещала людям Украины, что мы будем с вами столько, сколько нужно, и я хочу сдержать это обещание. Неважно, что думают мои немецкие избиратели""","Бербок: Я обещала людям Украины, что мы будем с вами столько, сколько нужно","((Бербок, Бербок), (Украины, Украина), (МИД Германии, МИД Германия), (МИД, МИД), (Германии, Германия))","((Бербок, Бербок), (Украины, Украина))"
109813,2021-09-20T11:25:08,"Думские выборы прошли в соответствии с российскими законами и демократическими стандартами, были открытыми и конкурентными, заявила миссия наблюдателей СНГ",миссия наблюдателей СНГ: Думские выборы прошли в соответствии с законами и демократическими стандартами,"((СНГ, СНГ),)","((СНГ, СНГ), (Думские выборы, Думские выборы))"
2642,2019-10-10T02:13:32,"Россиянка Юзик, задержанная ранее в Иране и вылетевшая 10 октября в Москву, чувствует себя нормально - посольство РФ в Тегеране РИА Новости",Посольство РФ: задержанная в Иране россиянка чувствует себя нормально,"((Юзик, Юзик), (РИА Новости, РИА Новости), (РФ, РФ), (Москву, Москва), (Иране, Иран), (Тегеране, Тегеран))","((РФ, РФ), (Иране, Иран))"
137131,2022-02-24T15:35:07,"Некоторые украинские военные в ДНР сдаются, увидев бегство своих командиров, сказал РИА Новости Басурин","Басурин: Некоторые украинские военные в ДНР сдаются, увидев бегство своих командиров","((РИА Новости, РИА Новости), (РИА Новости Басурин, РИА Новости Басурин), (Басурин, Басурин), (ДНР, ДНР))","((Басурин, Басурин), (ДНР, ДНР))"
86234,2021-05-14T11:26:10,"Крыша 8-этажного жилого дома горит в Екатеринбурге (Шейнкмана 19), идет эвакуация жильцов",Крыша 8-этажного жилого дома горит в Екатеринбурге (Шейнкмана 19),"((Екатеринбурге, Екатеринбург), (Шейнкмана, Шейнкмана))","((Екатеринбурге, Екатеринбург), (Шейнкмана, Шейнкмана))"
...,...,...,...,...,...
10160,2020-02-26T16:47:59,"Вопрос о переносе ПМЭФ из-за коронавируса пока не обсуждается, сообщила Голикова",Голикова: вопрос о переносе ПМЭФ из-за коронавируса пока не обсуждается,"((Голикова, Голикова), (ПМЭФ, ПМЭФ))","((Голикова, Голиков), (ПМЭФ, ПМЭФ))"
113889,2021-10-13T19:43:55,"Половина французов ставит Макрону ""двойку"": 52% считают его действия на посту президента неудовлетворительными, 22% одобряют его работу, а 26% говорят, что еще рано давать оценки, свидетельствует опрос Elabe для BFM TV","Половина французов ставит Макрону ""двойку""","((Макрону, Макрон), (BFM TV, BFM TV), (Elabe, Elabe))","((Макрону, Макрон),)"
124828,2021-12-21T14:02:55,"Шойгу: в России создан первый авиаполк истребителей МиГ-31 с гиперзвуковыми ракетами ""Кинжал""","В России создан первый авиаполк истребителей МиГ-31 с гиперзвуковыми ракетами ""Кинжал""","((Шойгу, Шойгу), (России, Россия), (Кинжал, Кинжал))","((России, Россия), (Кинжал, Кинжал))"
148012,2022-04-18T05:01:24,"ВВП Китая в I квартале 2022 вырос на 4,8%, сообщило статбюро. Власти ожидают, что темпы роста по итогам года составят около 5,5%.\n\nПо итогам 2021, несмотря на многочисленные локальные вспышки коронавируса, проблемы на рынке недвижимости и энергетический кризис, ВВП Китая вырос на 8,1% выше официального прогноза в 6%.","ВВП Китая в I квартале 2022 вырос на 4,8%","((Китая, Китай),)","((Китая, Китай),)"


In [11]:
df_sample["ners_nuniq_full"] = df_sample.ners_norm_full.map(lambda x: len(set(map(lambda y: y[1], x))))
df_sample["ners_nuniq_summ"] = df_sample.ners_norm_summ.map(lambda x: len(set(map(lambda y: y[1], x))))

In [12]:
df_sample.describe()

,ners_nuniq_full,ners_nuniq_summ
count,1000.000000,1000.000000
mean,2.865000,1.794000
std,2.285169,1.122865
min,0.000000,0.000000
25%,1.000000,1.000000
50%,2.000000,2.000000
75%,4.000000,2.000000
max,34.000000,8.000000


In [13]:
print("Процент новостей (исходных), в которых ко-во ner >= 2шт.: {}%".format(
       round(sum(df_sample.ners_nuniq_full >=2) / len(df_sample) * 100)))
print("Процент новостей (суммаризованых), в которых ко-во ner >= 2шт.: {}%".format(
       round(sum(df_sample.ners_nuniq_summ >=2) / len(df_sample) * 100)))

Процент новостей (исходных), в которых ко-во ner >= 2шт.: 74%
Процент новостей (суммаризованых), в которых ко-во ner >= 2шт.: 52%


In [14]:
df_sample.to_csv("sample_ner_norm.csv")
df_sample.to_excel("sample_ner_norm.xlsx")

## NER выбор варинта построения триплетов
Рассматриваются варианты:
1. Без привязки к subj/obj (т.е. ребра в триплетах не направленные, либо направление не зависит от subj/obj вершин):
    * Источник - полная новость, NER >= 2, без привязки к subj/obj
    * Источник - суммаризованная новость, NER >= 2, без привязки к subj/obj
2. С привязкой к subj/obj (т.е. мы отбираем только новости, в которых хотя бы одна из NER = subj, и хотя бы одна из NER = obj, ребра направленные от subj к obj, все ner, для которых не удалось определить принадлежность к subj/obj выкидываем из графа):
    * Источник - полная новость, NER >= 2, с привязкой к subj/obj
    * Источник - суммаризованная новость, NER >= 2, с привязкой к subj/obj
    
Предположительно основной критерий выбора - направленный граф с привязкой к subj/obj, лучше ненаправленного, но только в случае, достаточного количества (процента) новостей, которые мы сможем конвертировать в подобные триплеты. При этом, может быть рассмотрен вариант, что в бейзлайне используем самое простое решение без subj/obj (но сохраняем в базе триплеты, по 2 для каждой пары сущностей), с учетом возможного расширения функционала в будущем до subj/obj триплетов.

### Вариант 1

In [15]:
import itertools

# количество триплетов для Варианта 1. Без привязки к subj/obj (нам подойдут только новости c кол-во триплетов > 0)
df_sample["n_trip_nosobj_full"] = df_sample.ners_nuniq_full.map(lambda x: len(list(itertools.combinations(range(x), 2))))
df_sample["n_trip_nosobj_summ"] = df_sample.ners_nuniq_summ.map(lambda x: len(list(itertools.combinations(range(x), 2))))

In [16]:
df_sample["trip_nosobj_full"] = df_sample.apply(lambda x: list(itertools.combinations(list(set([pair[1] for pair in x.ners_norm_full])), 2)), axis=1)
df_sample["trip_nosobj_summ"] = df_sample.apply(lambda x: list(itertools.combinations(list(set([pair[1] for pair in x.ners_norm_summ])), 2)), axis=1)

In [17]:
df_sample

,date,text,summary,ners_norm_full,ners_norm_summ,ners_nuniq_full,ners_nuniq_summ,n_trip_nosobj_full,n_trip_nosobj_summ,trip_nosobj_full,trip_nosobj_summ
165028,2022-08-31T18:03:20,"Глава МИД Германии Бербок: ""Я обещала людям Украины, что мы будем с вами столько, сколько нужно, и я хочу сдержать это обещание. Неважно, что думают мои немецкие избиратели""","Бербок: Я обещала людям Украины, что мы будем с вами столько, сколько нужно","((Бербок, Бербок), (Украины, Украина), (МИД Германии, МИД Германия), (МИД, МИД), (Германии, Германия))","((Бербок, Бербок), (Украины, Украина))",5,2,10,1,"[(МИД, МИД Германия), (МИД, Украина), (МИД, Бербок), (МИД, Германия), (МИД Германия, Украина), (МИД Германия, Бербок), (МИД Германия, Германия), (Украина, Бербок), (Украина, Германия), (Бербок, Германия)]","[(Украина, Бербок)]"
109813,2021-09-20T11:25:08,"Думские выборы прошли в соответствии с российскими законами и демократическими стандартами, были открытыми и конкурентными, заявила миссия наблюдателей СНГ",миссия наблюдателей СНГ: Думские выборы прошли в соответствии с законами и демократическими стандартами,"((СНГ, СНГ),)","((СНГ, СНГ), (Думские выборы, Думские выборы))",1,2,0,1,[],"[(СНГ, Думские выборы)]"
2642,2019-10-10T02:13:32,"Россиянка Юзик, задержанная ранее в Иране и вылетевшая 10 октября в Москву, чувствует себя нормально - посольство РФ в Тегеране РИА Новости",Посольство РФ: задержанная в Иране россиянка чувствует себя нормально,"((Юзик, Юзик), (РИА Новости, РИА Новости), (РФ, РФ), (Москву, Москва), (Иране, Иран), (Тегеране, Тегеран))","((РФ, РФ), (Иране, Иран))",6,2,15,1,"[(Иран, Москва), (Иран, РИА Новости), (Иран, РФ), (Иран, Юзик), (Иран, Тегеран), (Москва, РИА Новости), (Москва, РФ), (Москва, Юзик), (Москва, Тегеран), (РИА Новости, РФ), (РИА Новости, Юзик), (РИА Новости, Тегеран), (РФ, Юзик), (РФ, Тегеран), (Юзик, Тегеран)]","[(Иран, РФ)]"
137131,2022-02-24T15:35:07,"Некоторые украинские военные в ДНР сдаются, увидев бегство своих командиров, сказал РИА Новости Басурин","Басурин: Некоторые украинские военные в ДНР сдаются, увидев бегство своих командиров","((РИА Новости, РИА Новости), (РИА Новости Басурин, РИА Новости Басурин), (Басурин, Басурин), (ДНР, ДНР))","((Басурин, Басурин), (ДНР, ДНР))",4,2,6,1,"[(Басурин, РИА Новости Басурин), (Басурин, РИА Новости), (Басурин, ДНР), (РИА Новости Басурин, РИА Новости), (РИА Новости Басурин, ДНР), (РИА Новости, ДНР)]","[(Басурин, ДНР)]"
86234,2021-05-14T11:26:10,"Крыша 8-этажного жилого дома горит в Екатеринбурге (Шейнкмана 19), идет эвакуация жильцов",Крыша 8-этажного жилого дома горит в Екатеринбурге (Шейнкмана 19),"((Екатеринбурге, Екатеринбург), (Шейнкмана, Шейнкмана))","((Екатеринбурге, Екатеринбург), (Шейнкмана, Шейнкмана))",2,2,1,1,"[(Шейнкмана, Екатеринбург)]","[(Шейнкмана, Екатеринбург)]"
...,...,...,...,...,...,...,...,...,...,...,...
10160,2020-02-26T16:47:59,"Вопрос о переносе ПМЭФ из-за коронавируса пока не обсуждается, сообщила Голикова",Голикова: вопрос о переносе ПМЭФ из-за коронавируса пока не обсуждается,"((Голикова, Голикова), (ПМЭФ, ПМЭФ))","((Голикова, Голиков), (ПМЭФ, ПМЭФ))",2,2,1,1,"[(Голикова, ПМЭФ)]","[(ПМЭФ, Голиков)]"
113889,2021-10-13T19:43:55,"Половина французов ставит Макрону ""двойку"": 52% считают его действия на посту президента неудовлетворительными, 22% одобряют его работу, а 26% говорят, что еще рано давать оценки, свидетельствует опрос Elabe для BFM TV","Половина французов ставит Макрону ""двойку""","((Макрону, Макрон), (BFM TV, BFM TV), (Elabe, Elabe))","((Макрону, Макрон),)",3,1,3,0,"[(BFM TV, Elabe), (BFM TV, Макрон), (Elabe, Макрон)]",[]
124828,2021-12-21T14:02:55,"Шойгу: в России создан первый авиаполк истребителей МиГ-31 с гиперзвуковыми ракетами ""Кинжал""","В России создан первый авиаполк истребителей МиГ-31 с гиперзвуковыми ракетами ""Кинжал""","((Шойгу, Шойгу), (России, Россия), (Кинжал, Кинжал))","((России, Россия), (Кинжал, Кинжал))",3,2,3,1,"[(Россия, Шойгу), (Россия, Кинжал), (Шойгу, Кинжал)]","[(Россия, Кинжал)]"
148012,2022-04-18T05:

### Вариант 2

In [18]:
!pip -q install stanza

In [19]:
import stanza
# nlp_stanza = stanza.Pipeline(lang='ru')
nlp_stanza = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma,ner,depparse')

In [20]:
triplet_pattern = {
    "subj": ["nsubj", "nsubj:pass", "csubj", "csubj:pass"],
    "obj": ["obj", "iobj"],
}

def get_subjobj_ent_graph_nodes(text: str, ents: dict):
    doc = nlp_stanza(text)
    subj_list = []
    obj_list = []
    # добавляем в целевые только ner, для которых найдены subj и obj в пределах одного предложения,
    # иначе, не добавляем, например, в рамках предложения найден только subj ner или только obj ner
    for sent in doc.sentences:
        tmp_subj_list = []
        tmp_obj_list = []
        
        for word in sent.words:
            if word.deprel in triplet_pattern["subj"]:
                tmp_subj_list.extend([ent for ent in ents.keys() if word.text in ent])
            elif word.deprel in triplet_pattern["obj"]:
                tmp_obj_list.extend([ent for ent in ents.keys() if word.text in ent])
        
        # print(tmp_subj_list, tmp_obj_list)
        if tmp_subj_list != [] and tmp_obj_list != []:
            subj_list.extend(tmp_subj_list)
            obj_list.extend(tmp_obj_list)

    return list(set(subj_list)), list(set(obj_list))

In [21]:
# unit-test
sample = "Путин упразднил министерство по делам Северного Кавказа, его функции отошли МЭР - указ"
ents = (("Северного Кавказа", "Северный Кавказ"), ("МЭР", "МЭР"), ("Путин","Путин"))
doc = nlp_stanza(sample)
depend_list = []
for sent in doc.sentences: 
        for word in sent.words:
            depend_list.append({
             "id": word.id,
             "word": word.text,
             "head id": word.head,
             "head": sent.words[word.head-1].text if word.head > 0 else "root",
             "deprel": word.deprel,
             "upos": word.upos
            })
print(pd.DataFrame(depend_list).to_string(index=False))

get_subjobj_ent_graph_nodes(sample, dict(ents))

 id         word  head id         head deprel  upos
  1        Путин        2    упразднил  nsubj PROPN
  2    упразднил        0         root   root  VERB
  3 министерство        2    упразднил    obj  NOUN
  4           по        5        делам   case   ADP
  5        делам        3 министерство   nmod  NOUN
  6    Северного        7      Кавказа   amod PROPN
  7      Кавказа        5        делам   nmod PROPN
  8            ,       11       отошли  punct PUNCT
  9          его       10      функции    det   DET
 10      функции       11       отошли    obl  NOUN
 11       отошли        2    упразднил   conj  VERB
 12          МЭР       11       отошли  nsubj PROPN
 13            -       14         указ  punct PUNCT
 14         указ       12          МЭР  appos  NOUN


([], [])

In [22]:
df_sample_tmp = df_sample.sample(50)

In [23]:
%%time
df_sample_tmp["trip_sobj_summ"] = df_sample_tmp.apply(lambda x: 
                                                      get_subjobj_ent_graph_nodes(x.summary,
                                                                                  dict(x.ners_norm_summ)), 
                                                      axis=1)

CPU times: user 28.2 s, sys: 126 ms, total: 28.3 s
Wall time: 14.2 s


In [24]:
%%time
df_sample_tmp["trip_sobj_full"] = df_sample_tmp.apply(lambda x: 
                                                      get_subjobj_ent_graph_nodes(x.text,
                                                                                  dict(x.ners_norm_full)), 
                                                      axis=1)

CPU times: user 54.4 s, sys: 233 ms, total: 54.7 s
Wall time: 27.4 s


In [25]:
df_sample_tmp

,date,text,summary,ners_norm_full,ners_norm_summ,ners_nuniq_full,ners_nuniq_summ,n_trip_nosobj_full,n_trip_nosobj_summ,trip_nosobj_full,trip_nosobj_summ,trip_sobj_summ,trip_sobj_full
132872,2022-02-07T14:08:42,Премьер Австралии по итогам заседания комитета нацбезопасности призвал сограждан немедленно покинуть Украину,Премьер Австралии призвал сограждан немедленно покинуть Украину,"((Австралии, Австралия), (Украину, Украина))","((Австралии, Австралия), (Украину, Украина))",2,2,1,1,"[(Украина, Австралия)]","[(Украина, Австралия)]","([], [])","([], [])"
139082,2022-03-01T20:12:02,"Четыре человека погибли в ДНР, 23 пострадали из-за обстрелов со стороны украинских войск с начала суток, заявил Басурин",Число жертв в ДНР превысило 300 человек,"((Басурин, Басурин), (ДНР, ДНР))","((ДНР, ДНР),)",2,1,1,0,"[(Басурин, ДНР)]",[],"([], [])","([], [])"
63832,2021-01-12T23:15:01,"Трамп заявил, что никакого риска его отстранения по 25-й поправке к Конституции не существует","Трамп заявил, что никакого риска его отстранения по 25-й поправке к Конституции не существует","((Конституции, Конституция), (Трамп, Трамп))","((Конституции, Конституция), (Трамп, Трамп))",2,2,1,1,"[(Трамп, Конституция)]","[(Трамп, Конституция)]","([], [])","([], [])"
156572,2022-06-17T16:31:23,"Путин заявил, что надо пересмотреть основания для заключения предпринимателей под стражу и сроки предварительного следствия по их делам, призвал правительство и верховный суд подготовить поправки",Путин призвал пересмотреть основания для заключения предпринимателей под стражу,"((Путин, Путин),)","((Путин, Путин),)",1,1,0,0,[],[],"([], [])","([], [])"
21228,2020-04-24T13:53:37,Матвиенко: сроков окончания карантина по коронавирусу никто сейчас назвать не может,Матвиенко: сроки окончания карантина по коронавирусу никто сейчас назвать не может,"((Матвиенко, Матвиенко),)","((Матвиенко, Матвиенко),)",1,1,0,0,[],[],"([], [])","([], [])"
119853,2021-11-19T10:26:42,"Саакашвили обратился к США с просьбой о помощи \n\n""Если США не придут меня защитить, это будет ужасным сигналом для всех прозападных лидеров и народов"", - гласит записка, которую он передал через своих адвокатов.",Саакашвили обратился к США с просьбой о помощи,"((США, США), (Саакашвили, Саакашвили))","((США, США), (Саакашвили, Саакашвили))",2,2,1,1,"[(США, Саакашвили)]","[(США, Саакашвили)]","([], [])","([], [])"
19425,2020-04-16T15:04:02,"Украинская рада изменила регламент, теперь она сможет принять закон о запрете возвращения Приватбанка Коломойскому, на котором настаивает МВФ",Рада сможет принять закон о запрете возвращения Приватбанка Коломойскому,"((МВФ, МВФ), (Украинская рада, Украинская рада), (Приватбанка, Приватбанк), (Коломойскому, Коломойский), (Приватбанка Коломойскому, Приватбанк Коломойский))","((Коломойскому, Коломойский), (Приватбанка Коломойскому, Приватбанк Коломойский), (Приватбанка, Приватбанк), (Рада, Рада))",5,4,10,6,"[(Коломойский, Приватбанк Коломойский), (Коломойский, Приватбанк), (Коломойский, МВФ), (Коломойский, Украинская рада), (Приватбанк Коломойский, Приватбанк), (Приватбанк Коломойский, МВФ), (Приватбанк Коломойский, Украинская рада), (Приватбанк, МВФ), (Приватбанк, Украинская рада), (МВФ, Украинская рада)]","[(Приватбанк Коломойский, Рада), (Приватбанк Коломойский, Приватбанк), (Приватбанк Коломойский, Коломойский), (Рада, Приватбанк), (Рада, Коломойский), (Приватбанк, Коломойский)]","([], [])","([], [])"
132438,2022-02-04T03:25:02,"США вводят визовые рестрикции в отношении многочисленных представителей белорусских властей за ""экстерриториальное преследование диссидентов"", заявил госдеп","Госдеп: США вводят визовые рестрикции в отношении белорусов за ""экстерриториальное преследование диссидентов""","((США, США),)","((США, США), (Госдеп, Госдеп))",1,2,0,1,[],"[(США, Госдеп)]","([], [])","([], [])"
171294,2022-10-19T12:52:03,"Верховная рада одобрила создание фонда ликвидации последствий боевых действий, источником его наполнения будет национализированное имущество",Верховная рада одобрила созд

Краткие итоги по подходу subj/obj, получаемые через теги dependancy:
* количество новостей, которые подходят под заданные паттерны абсолютно недостаточно, очень часто ner тегируются паттернами дополнений, причем это происходит как для субъектов, так и для объектов (т.е. их нельзя добавить в какой то один из словарей subj/obj)
* необходимо проверить альтернативный вариант, а именн для предложений в который root является глаголом, все нер что слева - субъекты, справа - объекты
    

In [26]:
def get_graph_nodes_root_left_right(text: str, ents: dict):
    doc = nlp_stanza(text)
    subj_list = []
    obj_list = []

    # добавляем в целевые пары только ner, в пределах одного предложения
    for sent in doc.sentences:
        tmp_subj_list = []
        tmp_obj_list = []
        
        for word in sent.words:
            if word.deprel == "root":
                if word.upos == "VERB":
                    tmp_subj_list.extend([ent_norm for ent_nonorm, ent_norm in ents.items() 
                                          if ent_nonorm in sent.text[:word.start_char]])
                    tmp_obj_list.extend([ent_norm for ent_nonorm, ent_norm in ents.items() 
                                          if ent_nonorm in sent.text[word.end_char:]])
                break

        # print(tmp_subj_list, tmp_obj_list) 
        if tmp_subj_list != [] and tmp_obj_list != []:
            subj_list.extend(tmp_subj_list)
            obj_list.extend(tmp_obj_list)
    
    return list(itertools.product(list(set(subj_list)), list(set(obj_list))))

In [27]:
# unit-test
sample = "Путин упразднил министерство по делам Северного Кавказа, его функции отошли МЭР - указ"
ents = (("Северного Кавказа", "Северный Кавказ"), ("МЭР", "МЭР"), ("Путин","Путин"))
doc = nlp_stanza(sample)
depend_list = []
for sent in doc.sentences: 
        for word in sent.words:
            depend_list.append({
             "id": word.id,
             "word": word.text,
             "head id": word.head,
             "head": sent.words[word.head-1].text if word.head > 0 else "root",
             "deprel": word.deprel,
             "upos": word.upos
            })
print(pd.DataFrame(depend_list).to_string(index=False))

get_graph_nodes_root_left_right(sample, dict(ents))

 id         word  head id         head deprel  upos
  1        Путин        2    упразднил  nsubj PROPN
  2    упразднил        0         root   root  VERB
  3 министерство        2    упразднил    obj  NOUN
  4           по        5        делам   case   ADP
  5        делам        3 министерство   nmod  NOUN
  6    Северного        7      Кавказа   amod PROPN
  7      Кавказа        5        делам   nmod PROPN
  8            ,       11       отошли  punct PUNCT
  9          его       10      функции    det   DET
 10      функции       11       отошли    obl  NOUN
 11       отошли        2    упразднил   conj  VERB
 12          МЭР       11       отошли  nsubj PROPN
 13            -       14         указ  punct PUNCT
 14         указ       12          МЭР  appos  NOUN


[('Путин', 'Северный Кавказ'), ('Путин', 'МЭР')]

In [28]:
# df_sample = df_sample.sample(50)

In [29]:
%%time
df_sample["trip_sobj_lr_full"] = df_sample.apply(lambda x: 
                                                 get_graph_nodes_root_left_right(x.text, 
                                                                                 dict(x.ners_norm_full)), 
                                                 axis=1)

CPU times: user 18min 28s, sys: 5.08 s, total: 18min 33s
Wall time: 9min 18s


In [30]:
%%time
df_sample["trip_sobj_lr_summ"] = df_sample.apply(lambda x: 
                                                 get_graph_nodes_root_left_right(x.summary, 
                                                                                 dict(x.ners_norm_summ)), 
                                                 axis=1)

CPU times: user 9min 31s, sys: 2.82 s, total: 9min 34s
Wall time: 4min 48s


In [31]:
df_sample[df_sample.n_trip_nosobj_full > 0]

,date,text,summary,ners_norm_full,ners_norm_summ,ners_nuniq_full,ners_nuniq_summ,n_trip_nosobj_full,n_trip_nosobj_summ,trip_nosobj_full,trip_nosobj_summ,trip_sobj_lr_full,trip_sobj_lr_summ
165028,2022-08-31T18:03:20,"Глава МИД Германии Бербок: ""Я обещала людям Украины, что мы будем с вами столько, сколько нужно, и я хочу сдержать это обещание. Неважно, что думают мои немецкие избиратели""","Бербок: Я обещала людям Украины, что мы будем с вами столько, сколько нужно","((Бербок, Бербок), (Украины, Украина), (МИД Германии, МИД Германия), (МИД, МИД), (Германии, Германия))","((Бербок, Бербок), (Украины, Украина))",5,2,10,1,"[(МИД, МИД Германия), (МИД, Украина), (МИД, Бербок), (МИД, Германия), (МИД Германия, Украина), (МИД Германия, Бербок), (МИД Германия, Германия), (Украина, Бербок), (Украина, Германия), (Бербок, Германия)]","[(Украина, Бербок)]",[],[]
2642,2019-10-10T02:13:32,"Россиянка Юзик, задержанная ранее в Иране и вылетевшая 10 октября в Москву, чувствует себя нормально - посольство РФ в Тегеране РИА Новости",Посольство РФ: задержанная в Иране россиянка чувствует себя нормально,"((Юзик, Юзик), (РИА Новости, РИА Новости), (РФ, РФ), (Москву, Москва), (Иране, Иран), (Тегеране, Тегеран))","((РФ, РФ), (Иране, Иран))",6,2,15,1,"[(Иран, Москва), (Иран, РИА Новости), (Иран, РФ), (Иран, Юзик), (Иран, Тегеран), (Москва, РИА Новости), (Москва, РФ), (Москва, Юзик), (Москва, Тегеран), (РИА Новости, РФ), (РИА Новости, Юзик), (РИА Новости, Тегеран), (РФ, Юзик), (РФ, Тегеран), (Юзик, Тегеран)]","[(Иран, РФ)]","[(Иран, РФ), (Иран, Тегеран), (Иран, РИА Новости), (Юзик, РФ), (Юзик, Тегеран), (Юзик, РИА Новости), (Москва, РФ), (Москва, Тегеран), (Москва, РИА Новости)]",[]
137131,2022-02-24T15:35:07,"Некоторые украинские военные в ДНР сдаются, увидев бегство своих командиров, сказал РИА Новости Басурин","Басурин: Некоторые украинские военные в ДНР сдаются, увидев бегство своих командиров","((РИА Новости, РИА Новости), (РИА Новости Басурин, РИА Новости Басурин), (Басурин, Басурин), (ДНР, ДНР))","((Басурин, Басурин), (ДНР, ДНР))",4,2,6,1,"[(Басурин, РИА Новости Басурин), (Басурин, РИА Новости), (Басурин, ДНР), (РИА Новости Басурин, РИА Новости), (РИА Новости Басурин, ДНР), (РИА Новости, ДНР)]","[(Басурин, ДНР)]","[(ДНР, Басурин), (ДНР, РИА Новости Басурин), (ДНР, РИА Новости)]",[]
86234,2021-05-14T11:26:10,"Крыша 8-этажного жилого дома горит в Екатеринбурге (Шейнкмана 19), идет эвакуация жильцов",Крыша 8-этажного жилого дома горит в Екатеринбурге (Шейнкмана 19),"((Екатеринбурге, Екатеринбург), (Шейнкмана, Шейнкмана))","((Екатеринбурге, Екатеринбург), (Шейнкмана, Шейнкмана))",2,2,1,1,"[(Шейнкмана, Екатеринбург)]","[(Шейнкмана, Екатеринбург)]",[],[]
19880,2020-04-18T09:36:32,Собянин: для контроля соблюдения пропускного режима при поездках на автомобиле в Москве будут задействованы камеры фотовидеофиксации нарушений ПДД,Собянин: для контроля соблюдения пропускного режима при поездках на автомобиле в Москве будут задействованы камеры фотовидеофиксации нарушений ПДД,"((Собянин, Собянин), (Москве, Москва))","((Собянин, Собянин), (Москве, Москва))",2,2,1,1,"[(Москва, Собянин)]","[(Москва, Собянин)]",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56069,2020-11-23T20:15:54,"Байден назначит Блинкена госсекретарем, Линду Томас-Гринфилд - постпредом при ООН, а Джейка Салливана - советником по нацбезопасности, сообщил его штаб. \n\nЭкс-госсекретарю Джону Керри планируется поручить заниматься вопросами климата.","Байден назначил Блинкена госсекретарем, Салливана - советником по нацбезопасности","((Джону Керри, Джон Керри), (Байден, Байден), (Джейка Салливана, Джейк Салливан), (Линду Томас-Гринфилд, Линда Томас-Гринфилд), (Блинкена, Блинкен), (ООН, ООН))","((Салливана, Салливан), (Байден, Байден), (Блинкена, Блинкен))",6,3,15,3,"[(ООН, Блинкен), (ООН, Джейк Салливан), (ООН, Байден), (ООН, Джон Керри), (ООН, Линда Томас-Гринфилд), (Блинкен, Джейк Салливан), (Блинкен, Байден), (Блинкен, Джон Керри), (Блинкен, Линда Томас-Гринфилд), (Джейк Салливан

In [32]:
df_sample["n_trip_sobj_lr_full"] = df_sample.trip_sobj_lr_full.map(len)
df_sample["n_trip_sobj_lr_summ"] = df_sample.trip_sobj_lr_summ.map(len)

In [33]:
df_sample

,date,text,summary,ners_norm_full,ners_norm_summ,ners_nuniq_full,ners_nuniq_summ,n_trip_nosobj_full,n_trip_nosobj_summ,trip_nosobj_full,trip_nosobj_summ,trip_sobj_lr_full,trip_sobj_lr_summ,n_trip_sobj_lr_full,n_trip_sobj_lr_summ
165028,2022-08-31T18:03:20,"Глава МИД Германии Бербок: ""Я обещала людям Украины, что мы будем с вами столько, сколько нужно, и я хочу сдержать это обещание. Неважно, что думают мои немецкие избиратели""","Бербок: Я обещала людям Украины, что мы будем с вами столько, сколько нужно","((Бербок, Бербок), (Украины, Украина), (МИД Германии, МИД Германия), (МИД, МИД), (Германии, Германия))","((Бербок, Бербок), (Украины, Украина))",5,2,10,1,"[(МИД, МИД Германия), (МИД, Украина), (МИД, Бербок), (МИД, Германия), (МИД Германия, Украина), (МИД Германия, Бербок), (МИД Германия, Германия), (Украина, Бербок), (Украина, Германия), (Бербок, Германия)]","[(Украина, Бербок)]",[],[],0,0
109813,2021-09-20T11:25:08,"Думские выборы прошли в соответствии с российскими законами и демократическими стандартами, были открытыми и конкурентными, заявила миссия наблюдателей СНГ",миссия наблюдателей СНГ: Думские выборы прошли в соответствии с законами и демократическими стандартами,"((СНГ, СНГ),)","((СНГ, СНГ), (Думские выборы, Думские выборы))",1,2,0,1,[],"[(СНГ, Думские выборы)]",[],[],0,0
2642,2019-10-10T02:13:32,"Россиянка Юзик, задержанная ранее в Иране и вылетевшая 10 октября в Москву, чувствует себя нормально - посольство РФ в Тегеране РИА Новости",Посольство РФ: задержанная в Иране россиянка чувствует себя нормально,"((Юзик, Юзик), (РИА Новости, РИА Новости), (РФ, РФ), (Москву, Москва), (Иране, Иран), (Тегеране, Тегеран))","((РФ, РФ), (Иране, Иран))",6,2,15,1,"[(Иран, Москва), (Иран, РИА Новости), (Иран, РФ), (Иран, Юзик), (Иран, Тегеран), (Москва, РИА Новости), (Москва, РФ), (Москва, Юзик), (Москва, Тегеран), (РИА Новости, РФ), (РИА Новости, Юзик), (РИА Новости, Тегеран), (РФ, Юзик), (РФ, Тегеран), (Юзик, Тегеран)]","[(Иран, РФ)]","[(Иран, РФ), (Иран, Тегеран), (Иран, РИА Новости), (Юзик, РФ), (Юзик, Тегеран), (Юзик, РИА Новости), (Москва, РФ), (Москва, Тегеран), (Москва, РИА Новости)]",[],9,0
137131,2022-02-24T15:35:07,"Некоторые украинские военные в ДНР сдаются, увидев бегство своих командиров, сказал РИА Новости Басурин","Басурин: Некоторые украинские военные в ДНР сдаются, увидев бегство своих командиров","((РИА Новости, РИА Новости), (РИА Новости Басурин, РИА Новости Басурин), (Басурин, Басурин), (ДНР, ДНР))","((Басурин, Басурин), (ДНР, ДНР))",4,2,6,1,"[(Басурин, РИА Новости Басурин), (Басурин, РИА Новости), (Басурин, ДНР), (РИА Новости Басурин, РИА Новости), (РИА Новости Басурин, ДНР), (РИА Новости, ДНР)]","[(Басурин, ДНР)]","[(ДНР, Басурин), (ДНР, РИА Новости Басурин), (ДНР, РИА Новости)]",[],3,0
86234,2021-05-14T11:26:10,"Крыша 8-этажного жилого дома горит в Екатеринбурге (Шейнкмана 19), идет эвакуация жильцов",Крыша 8-этажного жилого дома горит в Екатеринбурге (Шейнкмана 19),"((Екатеринбурге, Екатеринбург), (Шейнкмана, Шейнкмана))","((Екатеринбурге, Екатеринбург), (Шейнкмана, Шейнкмана))",2,2,1,1,"[(Шейнкмана, Екатеринбург)]","[(Шейнкмана, Екатеринбург)]",[],[],0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10160,2020-02-26T16:47:59,"Вопрос о переносе ПМЭФ из-за коронавируса пока не обсуждается, сообщила Голикова",Голикова: вопрос о переносе ПМЭФ из-за коронавируса пока не обсуждается,"((Голикова, Голикова), (ПМЭФ, ПМЭФ))","((Голикова, Голиков), (ПМЭФ, ПМЭФ))",2,2,1,1,"[(Голикова, ПМЭФ)]","[(ПМЭФ, Голиков)]","[(ПМЭФ, Голикова)]",[],1,0
113889,2021-10-13T19:43:55,"Половина французов ставит Макрону ""двойку"": 52% считают его действия на посту президента неудовлетворительными, 22% одобряют его работу, а 26% говорят, что еще рано давать оценки, свидетельствует опрос Elabe для BFM TV","Половина французов ставит Макрону ""двойку""","((Макрону, Макрон), (BFM TV, BFM TV), (Elabe, Elabe))","((Макрону, Макрон),)",3,1,3,0,"[(BFM TV, Elabe), (BFM TV, Макрон), (Elabe, Макрон)]",[],[],[],0,0
124828,2021-12-

In [34]:
print("Процент новостей, для которых мы можем сформировать триплеты.")
print("Источник - полная новость, NER >= 2, без привязки к subj/obj: {}%".format(
       round(sum(df_sample.n_trip_nosobj_full > 0) / len(df_sample) * 100)))
print("Источник - суммаризованная новость, NER >= 2, без привязки к subj/obj: {}%".format(
       round(sum(df_sample.n_trip_nosobj_summ > 0) / len(df_sample) * 100)))
print("Источник - полная новость, NER >= 2, с привязкой к subj/obj: {}%".format(
       round(sum(df_sample.n_trip_sobj_lr_full > 0) / len(df_sample) * 100)))
print("Источник - суммаризованная новость, NER >= 2, с привязкой к subj/obj: {}%".format(
       round(sum(df_sample.n_trip_sobj_lr_summ > 0) / len(df_sample) * 100)))

Процент новостей, для которых мы можем сформировать триплеты.
Источник - полная новость, NER >= 2, без привязки к subj/obj: 74%
Источник - суммаризованная новость, NER >= 2, без привязки к subj/obj: 52%
Источник - полная новость, NER >= 2, с привязкой к subj/obj: 46%
Источник - суммаризованная новость, NER >= 2, с привязкой к subj/obj: 30%


In [35]:
df_sample.to_csv("sample_graph_nodes.csv")
df_sample.to_excel("sample_graph_nodes.xlsx")

**Итоги выбора варинта построения триплетов.**

Решено использовать комбинированный подход без привязки к subj/obj, а именно:
* пытаемся получить набор вершин из NER на основе саммаризованной новости, если удается (т.е. для случая NER >= 2), то останавливаемся на этом, 
* если нет, пытаемся сформировать набор вершин из NER исходной не обработанной новости.

### Entity linking

In [36]:
# list(df_sample.normalize_ners)

In [37]:
ners = []
for row in list(df_sample.ners_norm_full):
    ners.extend(list(row))
ners_norm = list(map(lambda x: x[1], ners))

In [38]:
print(f"Before unique: {len(ners_norm)}")
ners_norm = list(set(ners_norm))
print(f"After uniaue: {len(ners_norm)}")

Before unique: 2891
After uniaue: 1270


In [39]:
import requests

session = requests.Session()
URL = 'https://www.wikidata.org/w/api.php'

def wbsearchentities(name):
    '''
    Docs: https://www.wikidata.org/w/api.php?action=help&modules=wbsearchentities
    '''
    res = session.post(URL, data={
        'action': 'wbsearchentities',
        'search': name,
        'language':'ru',
        'limit': '1',
        'format': 'json',
    })
    try:
        res_json = res.json()['search'][0]
    except:
        res_json = None
    return res_json


def wbgetentities_label(q_id):
    '''
    Docs: https://www.wikidata.org/w/api.php?action=help&modules=wbgetentities
    API позволяет делать запрос на множественные ID, возможно исользовать в будущем
    '''
    res = session.post(URL, data={
        'action': 'wbgetentities',
        'ids': q_id,
        'props': 'labels',
        'languages':'ru',
        'format': 'json',
    })
    try:
        res_json = res.json()['entities'][q_id]["labels"]['ru']['value']
    except:
        res_json = None
    return res_json

In [40]:
Q_id = wbsearchentities("ГД РФ")
Q_id

{'id': 'Q715641',
 'title': 'Q715641',
 'pageid': 672966,
 'display': {'label': {'value': 'State Duma', 'language': 'en'},
  'description': {'value': 'lower house of Federal Assembly of Russia',
   'language': 'en'}},
 'repository': 'wikidata',
 'url': '//www.wikidata.org/wiki/Q715641',
 'concepturi': 'http://www.wikidata.org/entity/Q715641',
 'label': 'State Duma',
 'description': 'lower house of Federal Assembly of Russia',
 'match': {'type': 'alias', 'language': 'ru', 'text': 'ГД РФ'},
 'aliases': ['ГД РФ']}

In [41]:
Q_id = wbsearchentities("Байконур")
Q_id

{'id': 'Q177477',
 'title': 'Q177477',
 'pageid': 177044,
 'display': {'label': {'value': 'Baikonur Cosmodrome', 'language': 'en'},
  'description': {'value': 'cosmodrome in Kazakhstan, used by Russia',
   'language': 'en'}},
 'repository': 'wikidata',
 'url': '//www.wikidata.org/wiki/Q177477',
 'concepturi': 'http://www.wikidata.org/entity/Q177477',
 'label': 'Baikonur Cosmodrome',
 'description': 'cosmodrome in Kazakhstan, used by Russia',
 'match': {'type': 'label', 'language': 'ru', 'text': 'Байконур'},
 'aliases': ['Байконур']}

In [42]:
Q_id = wbsearchentities("«Северный поток»")
Q_id

{'id': 'Q98978140',
 'title': 'Q98978140',
 'pageid': 97104376,
 'display': {'label': {'value': '«Северный поток» будет перекрыт на неопределенное время в плановых целях',
   'language': 'ru'},
  'description': {'value': 'Wikinews article', 'language': 'en'}},
 'repository': 'wikidata',
 'url': '//www.wikidata.org/wiki/Q98978140',
 'concepturi': 'http://www.wikidata.org/entity/Q98978140',
 'label': '«Северный поток» будет перекрыт на неопределенное время в плановых целях',
 'description': 'Wikinews article',
 'match': {'type': 'label',
  'language': 'ru',
  'text': '«Северный поток» будет перекрыт на неопределенное время в плановых целях'}}

In [43]:
Q_id = wbsearchentities("Парламент Великобритан")
Q_id

{'id': 'Q11010',
 'title': 'Q11010',
 'pageid': 12473,
 'display': {'label': {'value': 'Parliament of the United Kingdom',
   'language': 'en'},
  'description': {'value': 'supreme legislative body of the United Kingdom',
   'language': 'en'}},
 'repository': 'wikidata',
 'url': '//www.wikidata.org/wiki/Q11010',
 'concepturi': 'http://www.wikidata.org/entity/Q11010',
 'label': 'Parliament of the United Kingdom',
 'description': 'supreme legislative body of the United Kingdom',
 'match': {'type': 'label',
  'language': 'ru',
  'text': 'Парламент Великобритании'},
 'aliases': ['Парламент Великобритании']}

In [44]:
wbgetentities_label("Q715641")

'Государственная дума'

In [45]:
from tqdm import tqdm

In [46]:
%%time
ners_link = {k: wbsearchentities(k) for k in tqdm(ners_norm)}

df_ners_link = pd.DataFrame({"id": {k: v['id'] if v is not None else np.NaN 
                                    for k,v in ners_link.items()}}).reset_index().rename(columns={"index": "name"})
df_ners_link

100%|██████████| 1270/1270 [01:38<00:00, 12.86it/s]

CPU times: user 5.44 s, sys: 470 ms, total: 5.91 s
Wall time: 1min 38s


,name,id
0,"""Автозаводская""",NaN
1,"""Ильюшин""",NaN
2,"""Красноярск""",NaN
3,"""Одноклассники""",Q98977894
4,"""Орехово""",NaN
...,...,...
1265,украинские,Q2635283
1266,украинцы,Q44806
1267,улица Плеханов,Q4364937
1268,университет Хопкинс,Q193727


In [47]:
df_ners_link.describe()

,name,id
count,1270,976
unique,1270,928
top,"""Автозаводская""",Q244165
freq,1,3


In [48]:
df_ners_link[df_ners_link.duplicated("id", keep=False)].dropna().sort_values("id").head(50)

,name,id
695,Минобороны РФ,Q1146492
694,Минобороны,Q1146492
645,МО РФ,Q1146492
105,VK,Q116933
248,ВКонтакте,Q116933
253,ВС,Q132
94,Sun,Q132
270,Великобритания,Q145
239,Британия,Q145
484,КНР,Q148


In [49]:
%%time
df_ners_ids = df_ners_link.fillna("not_found").groupby("id", as_index=False).agg({"name": list})
df_ners_ids["count"] = df_ners_ids.name.map(len)
df_ners_ids.sort_values(by="count", ascending=False, inplace=True)
df_ners_ids["main_label"] = df_ners_ids.id.map(lambda x: wbgetentities_label(x) if x != "not_found" else np.NaN)
df_ners_ids

CPU times: user 2.94 s, sys: 169 ms, total: 3.11 s
Wall time: 1min 6s


,id,name,count,main_label
928,not_found,"[""Автозаводская"", ""Ильюшин"", ""Красноярск"", ""Орехово"", ""Ржавчина"", ""Сарматы"", ""Циркон"", ANIX Group, Adrenaline Stadium, ArcelorMittal Кривой Рог, CAS по Валиевой, EMP F2, Facebook Цукерберг, Mission Extension Vehicle-1, Pussy Riot Алехина, RT Франция, Services Globalization Index, Sputnik Казахстан, SvetoCopy, Sylvamo, TPQ-36, WSJ), А321, АП Козак, АЭС ""Аккую, АЭС ""Аккую"", Алексей Спасский, Альфа-Капиталь, Ан-26 воздушные силы Украина, Андрей Ромашов, Анти-Россия, Аркадий Ротенбергов, Армянское МО, Артемовск Донецкая Народная Республика, Артемовском, Асад Шаабан, Аскар Умаров, Афимолл, Банк Англия, Бастрыкин - Кремль, Баштанки, Беларуськалия, Белорусская нефтяная компания, Белорусские правозащитники, Белтелерадиокомпании, Большой террор 1937, Борис Москвич, Бюджетникам, ВМС Франция, ВНИИ ГОЧС, ВС Армения, ВС РФ, Валентина Каминская, Валиевой, Вардениса, Ваффен-СС"", Верховная рада Разумков, Верховный суд Россия, Весенняя сессия, Виктор Филинков, Владимир Габриелян, Владимир Джабаров, Владлен Хасиков, Внимание всем!, Волгонефть ВН-249, Вооруженные силы Турция, Всемирный банк по Россия, Вячеслав Цеповяз, ГУ МЧС, ГУМЧС, Германская BioNTech, Герои Россия, Глава министерство оборона Украина, Главврач больница, Главное Разведывательное Управление (ГРУ), Глуховского, Госдума Луговой, Госдума по СНГ, Госдумы Нифантьев, Госдумы Нифантьев (ЕР), Госкомитет Нацбезопасности, Госпогранкомитет, Госпогранкомитет Белоруссия, Государственная кабельная сеть, Градского, Губернский театр, Диана Маунтбеттен-Виндзор, Димитрис Ботинис, Договор по открытое небо, Думский комитет по труд, Дюпьи Де Лом, Евгений Блюм, Европейская ассоциация по туризм, Европейские биржи, Запорожья, Зеленский Нобелевская премия мир, Зинаида Батраева, Золгенсма, ИГ*, Ибице, ...]",294,NaN
805,Q7747,"[Владимир Владимирович, Владимир Путин, Путин]",3,Владимир Владимирович Путин
377,Q244165,"[Карабах, НКР, Нагорно-Карабахская республика]",3,Нагорно-Карабахская Республика
48,Q1146492,"[МО РФ, Минобороны, Минобороны РФ]",3,Министерство обороны Российской Федерации
158,Q159,"[РФ, Российская Федерация, Россия]",3,Россия
...,...,...,...,...
323,Q21453193,[Перминов],1,Перминов
324,Q21488251,[Абрамович],1,Абрамович
325,Q21488906,[Краснов],1,Краснов
326,Q21491544,[Деркач],1,Деркач


Промежуточные выводы и подзадачи (task): 
* в main_label встречается явная дичь типа Q98978140 "«Северный поток» будет перекрыт на неопределенное время в плановых целях", или Q4491700 "Кремль." = "Фонд эффективной политики":
    * выяснить действительно ли подобная дичь в основных лейблах викидаты ==> 
        * с северным потоком, такой результат мы получаем, если ищем с русскоязычными кавычками, т.е. с «», при этом мы получаем id статьи, отличительное поле для статей 'description': 'Wikinews article', если искать без кавычек, то находится верный id
        * так же множество терминов не находится по причине обрамления их обычными кавычками типа "", например '"Байконур"' не находится, а 'Байконур' прекрасно находится
        * в случае с "Кремль." мы получаем неверные данные изза "." в конце, если ее убрать, то получаем верный результат. Выяснить на каком этапе мы получаем неверную токенизацию или ner с "."
        == > task: нужно чистить ner от излишней пунктуации перед поиском (возможно использовать 2 варианта, первый раз почищенный запрос, если не находит, то исходный, хотя предварительно вообще провести анализ, есть ли примеры нахождения адекватных сущностей с кавычками)
    * в целом, относительно часто мы получаем очень длинные развернутые термины, которые скорей всего будут очень плох смотреться как вершины графа
    * по анализа вопросов выше, рассмотреть возможность определять основной лейбл статистически, по максимальному кол-ву использования в новостях
* поиск работает для урезанной фразы, но не работает для фраз с неверным окончанием, например wbsearchentities("Парламент Великобритан") находит сущность Парламент Великобритании, а wbsearchentities("Парламент Великобритания") нет ==> возможно является точкой роста качества в будущем

In [50]:
df_ners_ids["main_label"] = df_ners_ids.id.map(lambda x: wbgetentities_label(x) if x != "not_found" else np.NaN)

In [51]:
df_ners_ids.head(50)

,id,name,count,main_label
928,not_found,"[""Автозаводская"", ""Ильюшин"", ""Красноярск"", ""Орехово"", ""Ржавчина"", ""Сарматы"", ""Циркон"", ANIX Group, Adrenaline Stadium, ArcelorMittal Кривой Рог, CAS по Валиевой, EMP F2, Facebook Цукерберг, Mission Extension Vehicle-1, Pussy Riot Алехина, RT Франция, Services Globalization Index, Sputnik Казахстан, SvetoCopy, Sylvamo, TPQ-36, WSJ), А321, АП Козак, АЭС ""Аккую, АЭС ""Аккую"", Алексей Спасский, Альфа-Капиталь, Ан-26 воздушные силы Украина, Андрей Ромашов, Анти-Россия, Аркадий Ротенбергов, Армянское МО, Артемовск Донецкая Народная Республика, Артемовском, Асад Шаабан, Аскар Умаров, Афимолл, Банк Англия, Бастрыкин - Кремль, Баштанки, Беларуськалия, Белорусская нефтяная компания, Белорусские правозащитники, Белтелерадиокомпании, Большой террор 1937, Борис Москвич, Бюджетникам, ВМС Франция, ВНИИ ГОЧС, ВС Армения, ВС РФ, Валентина Каминская, Валиевой, Вардениса, Ваффен-СС"", Верховная рада Разумков, Верховный суд Россия, Весенняя сессия, Виктор Филинков, Владимир Габриелян, Владимир Джабаров, Владлен Хасиков, Внимание всем!, Волгонефть ВН-249, Вооруженные силы Турция, Всемирный банк по Россия, Вячеслав Цеповяз, ГУ МЧС, ГУМЧС, Германская BioNTech, Герои Россия, Глава министерство оборона Украина, Главврач больница, Главное Разведывательное Управление (ГРУ), Глуховского, Госдума Луговой, Госдума по СНГ, Госдумы Нифантьев, Госдумы Нифантьев (ЕР), Госкомитет Нацбезопасности, Госпогранкомитет, Госпогранкомитет Белоруссия, Государственная кабельная сеть, Градского, Губернский театр, Диана Маунтбеттен-Виндзор, Димитрис Ботинис, Договор по открытое небо, Думский комитет по труд, Дюпьи Де Лом, Евгений Блюм, Европейская ассоциация по туризм, Европейские биржи, Запорожья, Зеленский Нобелевская премия мир, Зинаида Батраева, Золгенсма, ИГ*, Ибице, ...]",294,NaN
805,Q7747,"[Владимир Владимирович, Владимир Путин, Путин]",3,Владимир Владимирович Путин
377,Q244165,"[Карабах, НКР, Нагорно-Карабахская республика]",3,Нагорно-Карабахская Республика
48,Q1146492,"[МО РФ, Минобороны, Минобороны РФ]",3,Министерство обороны Российской Федерации
158,Q159,"[РФ, Российская Федерация, Россия]",3,Россия
760,Q660770,"[РЖД, Российские]",2,Российские железные дороги
832,Q813,"[Киргизия, Кыргызстан]",2,Киргизия
375,Q242345,"[S&P, S&P 500]",2,S&P 500
853,Q84263196,"[COVID, COVID-19]",2,коронавирусная инфекция COVID-19
283,Q2012050,"[Донбасс, Донецкая область]",2,Донецкая область


In [52]:
df_ners_link.to_csv("df_ners_link.csv")
df_ners_link.to_excel("df_ners_link.xlsx")

df_ners_ids.to_csv("df_ners_ids.csv")
df_ners_ids.to_excel("df_ners_ids.xlsx")

In [53]:
# # PREFIX entity: <http://www.wikidata.org/entity/>
# # PREFIX wdt: <http://www.wikidata.org/prop/direct/>
# # SELECT ?syno
# # WHERE {
# #   ?O ?P ?id .
# #   OPTIONAL{?id skos:altLabel ?syno
# #       filter (lang(?syno) = 'ru')}
# #   VALUES ?id {entity:Q159}   
# #   SERVICE wikibase:label {bd:serviceParam wikibase:language "ru" .}}
# # GROUP BY ?syno
# # LIMIT 5000


# import sparql

# def create_query(first_id):
#     q = ('''
#     PREFIX entity: <http://www.wikidata.org/entity/>
#     PREFIX wdt: <http://www.wikidata.org/prop/direct/>
#     SELECT ?syno
#     WHERE {
#       ?O ?P ?id .
#       OPTIONAL{?id skos:altLabel ?syno
#           filter (lang(?syno) = 'ru')}
#       VALUES ?id {entity:'''+ first_id +'''}   
#       SERVICE wikibase:label {bd:serviceParam wikibase:language "ru" .}}''')
#     return q

# synonyms = []
# query = create_query(wbgetentities("Государственная дума"))
# result = sparql.query('https://query.wikidata.org/sparql', query)
# for r in result:
#     values = sparql.unpack_row(r)
#     if values[0] not in synonyms:
#            synonyms.append(values[0])
# print(synonyms)